In [1]:
# DROWSINESS DETECTION WITH OPENCV

# import the necessary packages

from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2

# SciPy package for computing the euclidean distance between the facial landmarks 
# points in the eye aspect ratio calculation

# Thread class so we can play our alarm in a seperate thread from the main thread
# to ensure our script doesn't pause execution while the alarm sounds

# In order to play our WAV/MP3 alarm, we need the playsound library, a pure Python,
# cross-platform implementation for playing simple sounds

# define the sound_alarm function which accepts a path to an audio file residing on disk
# and then plays the file

def sound_alarm(path):
    
    # play an alarm sound
    
    playsound.playsound(path)
    
# define the eye_aspect_ration function which computes the ratio of distances between the 
# vertical eye landmarks and the distances between the horizontal eye landmarks
    
def eye_aspect_ratio(eye):
    
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye
    # landmark (x, y)-coordinates
    
    C = dist.euclidean(eye[0], eye[3])
    
    # compute the eye aspect ratio
    
    ear = (A + B) / (2.0 * C)
    
    # return the eye aspect ratio
    
    return ear

# The return value of the eye aspect ratio will be approximately constant when the eye is open.
# The value will then rapid decrease towards zero during a blink.

# If the eye is closed, the eye aspect ratio will again remain approximately constant, but will
# be much smaller than the ratio when the eye is open.

# The eye aspect ratio is constant (indicating the eye is open), then rapidly drops to zero, 
# then increases again, indicating a blink has taken place.

# In our drowsiness detector case, we’ll be monitoring the eye aspect ratio to see if the value
# falls but does not increase again, thus implying that the person has closed their eyes.

# construct the argument parse and parse the arguments

# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--shape-predictor", required = True, help = "path to facial landmark predictor")
# ap.add_argument("-a", "--alarm", type = str, default = "", help = "path to alarm .WAV file")
# ap.add_argument("-w", "--webcam", type = int, default = 0, help = "index of webcam on system")
# args = vars(ap.parse_args())

# define two constants, one for the eye aspect ratio to indicate blink and then a second
# constant for the number of consecutive frames the eye must be below the threshold to set
# off the alarm

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 48

# EYE_AR_THRESH - If the eye aspect ratio falls below this threshold, we'll start counting
# the number of frames the person has closed their eyes for

# EYE_AR_CONSEC_FRAMES - If the number of frames the person has closed their eyes in exceeds
# EYE_AR_CONSEC FRAMES, we'll sound an alarm

# Setting EYE_AR_CONSEC_FRAMES to 48 means that if a person has closed their eyes for 48
# consecutive frames, we'll play the alarm sound. Decreasing this value makes the
# drowsiness detector more sensitive while increasing it makes it less sensitive

# initialize the frame counter as well as a boolean used to indicate if the alarm is
# going off

COUNTER = 0
ALARM_ON = False

# COUNTER - The total number of consecutive frames where the eye aspect ratio is below
# EYE_AR_THRESH

# If COUNTER exceeds EYE_AR_CONSEC_FRAMES, then we'll update the boolean ALARM_ON

# initialize dlib's face detector (HOG-based i.e. Histogram of Oriented Gradients-based)
# and then create the facial landmark predictor

print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()

# predictor = dlib.shape_predictor(args["shape_predictor"])

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# grab the indexes of the facial landmarks for the left and right eye, respectively

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# using these indexes, we'll be able to extract the eye regions via an array slice

# start the video stream thread

print("[INFO] starting video stream thread...")

# vs = VideoStream(src = args["webcam"]).start()

vs = VideoStream(src = 0).start() # for built-in webcam or USB camera
time.sleep(1.0) # pause for a second to allow the camera sensor to warm up

# loop over frames from the video stream

while True:
    
    # grab the frame from the threaded video file stream, resize it, and
    # convert it to grayscale channels (pre-processing)
    
    frame = vs.read()
    frame = imutils.resize(frame, width = 450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect face(s) in the grayscale frame (by applying dlib's face detector)
    
    rects = detector(gray, 0)
    
    # loop over the face detections
    
    for rect in rects:
    
        # determine the facial landmarks for the face region, then convert
        # the facial landmark (x, y)-coordinates to a NumPy array
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        # extract the left and right coordinates (using NumPy array slicing),
        # then use the coordinates to compute the eye aspect ratio for both eyes
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # average the eye aspect ratio together for both eyes
        
        ear = (leftEAR + rightEAR) / 2.0
        
        # In our implementation (specifically related to driver drowsiness)
        # we assume there is only one face - the driver, but the for loop
        # is just in case for application of this technique to videos with
        # more than one face
        
        # compute the convex hull for the left and right eye, then visualize
        # each of the eyes
        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # check to see if the eye aspect ratio is below the blink threshold,
        # and if so, increment the blink frame counter
        
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            
            # if the eyes were closed for a sufficient number of consecutive
            # frames, then sound the alarm
            
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                
                # if the alarm is not on, turn it on

                if not ALARM_ON:
                    
                    ALARM_ON = True
                    
                    # check to see if an alarm file was supplied, and if so, start
                    # a thread to have the alarm sound played in the background
                    
                    # if args["alarm"] != "":
                       # t = Thread(target = sound_alarm, args = (args["alarm"],))
                       # t.deamon = True
                       # t.start()
                    
                    # uncomment the above four lines when using argparse
                    
                    t = Thread(target = sound_alarm, args = ("C:/Users/Jahnavi/Downloads/mixkit-street-public-alarm-997.wav",))
                    t.deamon = True
                    t.start()
                    
                # draw an alarm on the frame
                
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
        # otherwise, the eye aspect ratio is not below the blink threshold,
        # so reset the counter and alarm
        
        else:
            COUNTER = 0
            ALARM_ON = False
            
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
    # show the frame
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the 'q' key was pressed, break from the loop
    
    if key == ord("q"):
        break
        
# do a bit of cleanup

cv2.destroyAllWindows()
vs.stop()

[INFO] loading facial landmark predictor...
[INFO] starting video stream thread...
